In [33]:
import pandas as pd
import numpy as np
from pytrends.request import TrendReq
from sklearn.preprocessing import MinMaxScaler

In [18]:
pytrends = TrendReq(hl='en-US', tz=360)
country_df = pd.read_csv('../country.csv', delimiter='\t')

In [75]:
kw_list = ["United States", "China", "France", "Ukraine", "Russia"] # country_df['name'].sample(5).to_list()
payload = pytrends.build_payload(kw_list, cat=0, timeframe='today 5-y', geo='US', gprop='')

In [77]:
series = pytrends.interest_over_time()[kw_list]
series[kw_list] = MinMaxScaler().fit_transform(series)

# Training Data

In [96]:
import argparse
import numpy as np
import os
import pandas as pd


def generate_graph_seq2seq_io_data(
        df, x_offsets, y_offsets
):
    num_samples, num_nodes = df.shape
    data = np.expand_dims(df.values, axis=-1)
    # epoch_len = num_samples + min(x_offsets) - max(y_offsets)
    x, y = [], []
    # t is the index of the last observation.
    min_t = abs(min(x_offsets))
    max_t = abs(num_samples - abs(max(y_offsets)))  # Exclusive
    for t in range(min_t, max_t):
        x_t = data[t + x_offsets, ...]
        y_t = data[t + y_offsets, ...]
        x.append(x_t)
        y.append(y_t)
    x = np.stack(x, axis=0)
    y = np.stack(y, axis=0)
    return x, y

In [101]:
def generate_train_val_test(df, input_length, output_length):

    x_offsets = np.sort(
            np.concatenate((np.arange(-input_length, 1, 1),))
        )
    y_offsets = np.sort(np.arange(1, output_length, 1))

    x, y = generate_graph_seq2seq_io_data(
        df,
        x_offsets=x_offsets,
        y_offsets=y_offsets,
        # add_time_in_day=True,
        # add_day_in_week=False,
    )

    print("x shape: ", x.shape, ", y shape: ", y.shape)
    # Write the data into npz file.
    # num_test = 6831, using the last 6831 examples as testing.
    # for the rest: 7/8 is used for training, and 1/8 is used for validation.
    num_samples = x.shape[0]
    num_test = round(num_samples * 0.2)
    num_train = round(num_samples * 0.7)
    num_val = num_samples - num_test - num_train

    # train
    x_train, y_train = x[:num_train], y[:num_train]
    # val
    x_val, y_val = (
        x[num_train: num_train + num_val],
        y[num_train: num_train + num_val],
    )
    # test
    x_test, y_test = x[-num_test:], y[-num_test:]

    return x_train, y_train, x_test, y_test

In [109]:
x_train, y_train, x_test, y_test = generate_train_val_test(series, 10, 10)

x shape:  (241, 11, 5, 1) , y shape:  (241, 9, 5, 1)


## Model

In [148]:
from torch_geometric_temporal.nn import MTGNN

model = MTGNN(
    gcn_true=True,
    build_adj=True,
    gcn_depth=2,
    num_nodes=5,
    seq_length=x_train.shape[1],
    kernel_set=None, 
    kernel_size=None, 
    dropout=0.3, 
    subgraph_size=20, 
    node_dim=40, 
    dilation_exponential=1, 
    conv_channels=32, 
    residual_channels=32, 
    skip_channels=64, 
    end_channels=128, 
    in_dim=2, 
    out_dim=12, 
    layers=3, 
    propalpha=0.05, 
    tanhalpha=3,
    layer_norm_affline=True
    )

TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'

In [ ]:
model()

In [130]:
device = torch.device('cuda:0')
num_nodes = x_train.shape[-2]
gcn_true = True
buildA_true = True #We want to build the Adjacency matrix.
gcn_depth = 8
seq_length = x_train.shape[1]
model = gtnet(buildA_true=buildA_true, gcn_true=gcn_true, gcn_depth=gcn_depth, num_nodes=num_nodes, device=device, seq_length=seq_length)

## Training pipeline

In [141]:
x_sample.shape

torch.Size([1, 5, 11, 1])

In [138]:
x_sample = torch.tensor(x_train[0]).unsqueeze(0).transpose(1,2)

In [145]:
start_conv = nn.Conv2d(in_channels=2,
                                    out_channels=12,
                                    kernel_size=(1, 1))

In [144]:
start_conv()

tensor([[[[0.0870],
          [0.0870],
          [0.0870],
          [0.0870],
          [0.0870],
          [0.0870],
          [0.1304],
          [0.1304],
          [0.0870],
          [0.0870],
          [0.0870]],

         [[0.0588],
          [0.0588],
          [0.0588],
          [0.1176],
          [0.0588],
          [0.0588],
          [0.0588],
          [0.0588],
          [0.0588],
          [0.0588],
          [0.0000]],

         [[0.0667],
          [0.1333],
          [0.0667],
          [0.0667],
          [0.0667],
          [0.0667],
          [0.0667],
          [0.0667],
          [0.0667],
          [0.0667],
          [0.0667]],

         [[0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000]],

         [[0.0294],
          [0.0147],
          [0.0147],
          [0.0147],
          [0.0588],
          [0